# IPO Portfolio Optimizer (WRDS)

Runs the GRU allocator on IPO data from **SDC New Deals** (all rows where ipodate is not null) + **Compustat** (daily open/close). Market data and shares from WRDS. SDC is the sole IPO source; no CSV fallback.

In [1]:
# Imports and path setup
import sys
from pathlib import Path

ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import torch

from src.wrds_data import get_connection, load_market_returns_wrds
from src.data_layer import align_returns, add_optional_features, build_rolling_windows, train_val_split
from src.train import run_training
from src.export import predict_weights, portfolio_stats, export_weights_csv, export_summary
from src.policy_layer import ipo_tilt_to_position_scale, policy_rule

In [2]:
# Connect to WRDS (set WRDS_USERNAME and WRDS_PASSWORD env vars for non-interactive)
conn = get_connection()
print("Connected to WRDS.")

Loading library list...


Done
Connected to WRDS.


## IPO Data: SDC New Deals + Compustat

IPO tickers come from **SDC New Deals** (all rows where ipodate is not null); daily open/close prices come from **Compustat** (comp.sec_dprc). SDC schema is `sdc`, table `wrds_ni_details`.

In [ ]:
# IPO data: SDC New Deals (all rows where ipodate is not null) + Compustat
from src.wrds_data import load_ipo_data_from_sdc_wrds

ipo_csv = load_ipo_data_from_sdc_wrds(conn, start='2024-01-01', end='2025-12-31', library='sdc')
print(f"IPO data from SDC + Compustat: {len(ipo_csv)} rows, {ipo_csv['tic'].nunique()} tickers")

ipo_csv['datadate'] = pd.to_datetime(ipo_csv['datadate'])
ipo_df = ipo_csv.groupby('tic').agg({'datadate': 'min'}).reset_index()
ipo_df.columns = ['ticker', 'ipo_date']
ipo_df = ipo_df.sort_values('ipo_date').reset_index(drop=True)

prices_ipo = ipo_csv.pivot_table(index='datadate', columns='tic', values='prccd')
prices_ipo.index = pd.to_datetime(prices_ipo.index).normalize()
start_d = prices_ipo.index.min().strftime('%Y-%m-%d')
end_d = prices_ipo.index.max().strftime('%Y-%m-%d')

print(f"IPO tickers: {len(ipo_df)}")
print(f"Date range: {start_d} to {end_d}")

In [3]:
# IPO data (ipo_csv, ipo_df, prices_ipo, start_d, end_d) loaded in previous cell

IPO tickers: 379
Date range: 2025-01-02 to 2025-12-31


In [4]:
# Custom market-cap weighted portfolio (replaces SPY) - Compustat, then CRSP, then CRSP index (no yfinance)
PORTFOLIO_TICKERS = [
    "AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "AVGO", "ORCL", "IBM", "CSCO",
    "JPM", "V", "MA", "GS", "MS", "BAC", "C", "AXP", "SCHW",
    "UNH", "JNJ", "LLY", "PFE", "MRNA", "BMY", "GILD", "CVS", "VRTX", "ISRG",
    "WMT", "NKE", "PG", "TGT", "COST", "KO", "PEP", "MCD", "SBUX", "YUM",
    "XOM", "CVX", "NEE", "DUK", "SO", "D", "ENB", "SLB", "EOG", "PSX",
    "DE", "LMT", "RTX", "BA", "CAT", "GE", "HON", "UPS", "EMR", "NOC",
    "PLD", "AMT", "EQIX", "O", "SPG", "VICI", "DLR", "WY", "EQR", "PSA",
    "TSLA", "FDX", "GM", "F", "RIVN", "NIO", "CSX", "UNP", "DAL",
    "TSM", "ASML", "AMD", "TXN", "INTC", "MU", "QCOM", "LRCX", "NXPI", "ADI",
    "ADM", "BG", "CF", "TSN", "MOS", "FMC", "CAG", "SYY", "HRL", "MDLZ",
    "NFLX", "DIS", "PARA", "WBD", "CMCSA", "SPOT", "LYV", "TTWO", "EA",
]
from src.wrds_data import load_market_returns_wrds, load_portfolio_returns_compustat_wrds, load_portfolio_returns_value_weighted_wrds, load_stock_returns_wrds
market_ret = load_portfolio_returns_compustat_wrds(conn, start=start_d, end=end_d, tickers=PORTFOLIO_TICKERS, value_weighted=True)
if market_ret.empty or len(market_ret.dropna()) < 50:
    portfolio_rets = load_stock_returns_wrds(conn, start=start_d, end=end_d, tickers=PORTFOLIO_TICKERS)
    if portfolio_rets.empty or portfolio_rets.shape[1] < 5:
        market_ret = load_market_returns_wrds(conn, start=start_d, end=end_d)
        print("Using CRSP market index (crsp.dsi)")
    else:
        market_ret = portfolio_rets.mean(axis=1).dropna()
        market_ret.name = "market_return"
        print("Using equal-weighted portfolio (CRSP)")
market_ret = market_ret.reindex(prices_ipo.index).dropna()
if len(market_ret) < 50:
    raise RuntimeError("Insufficient portfolio return data")

# Shares from comp.funda
ipo_tickers = ipo_df['ticker'].tolist()
gvkeys = ipo_csv[['tic', 'gvkey']].drop_duplicates()
gvkey_list = "','".join(gvkeys['gvkey'].astype(str).str.zfill(6).unique().tolist())
shares_df = conn.raw_sql(f"""
    select gvkey, datadate, csho
    from comp.funda
    where gvkey in ('{gvkey_list}')
        and datadate >= '2024-01-01'
        and csho > 0
        and indfmt = 'INDL' and datafmt = 'STD'
""", date_cols=['datadate'])

shares_outstanding = {}
if len(shares_df) > 0:
    last_csho = shares_df.sort_values('datadate').groupby('gvkey')['csho'].last()
    gvkey_to_tic = dict(zip(gvkeys['gvkey'].astype(str).str.zfill(6), gvkeys['tic']))
    for gvkey, csho in last_csho.items():
        t = gvkey_to_tic.get(str(gvkey).zfill(6))
        if t:
            shares_outstanding[t] = float(csho) * 1000

for t in ipo_tickers:
    if t in prices_ipo.columns and t not in shares_outstanding:
        p = prices_ipo[t].dropna()
        if len(p) > 0 and p.iloc[-1] > 0:
            shares_outstanding[t] = 1e6 / p.iloc[-1]

prices = prices_ipo.copy().ffill().bfill()

print(f"Market return days: {len(market_ret)} (market-cap weighted)")
print(f"Prices shape: {prices.shape}, Tickers with shares: {len(shares_outstanding)}")

Prices shape: (250, 380), Tickers with shares: 379


In [5]:
# Build 180-day market-cap weighted IPO index
def build_ipo_index_mcap(prices_df, ipo_dates_df, shares_dict, holding_days=180, min_names=1):
    ipo_lookup = dict(zip(ipo_dates_df['ticker'], ipo_dates_df['ipo_date']))
    returns_df = prices_df.pct_change()
    trading_days = {t: prices_df[t].dropna().index.tolist() for t in prices_df.columns 
                    if t != 'SPY' and t in ipo_lookup}
    all_dates = prices_df.index.tolist()
    index_data = []
    for date in all_dates:
        market_caps = {}
        for ticker, ipo_date in ipo_lookup.items():
            if ticker not in trading_days or ticker not in shares_dict:
                continue
            ticker_days = trading_days[ticker]
            first_trade_idx = next((i for i, d in enumerate(ticker_days) if d >= ipo_date), None)
            if first_trade_idx is None:
                continue
            if date in ticker_days:
                current_idx = ticker_days.index(date)
                if 0 <= current_idx - first_trade_idx < holding_days:
                    try:
                        cp = prices_df.loc[date, ticker]
                        if pd.notna(cp) and cp > 0:
                            market_caps[ticker] = cp * shares_dict[ticker]
                    except Exception:
                        pass
        total_mcap = sum(market_caps.values())
        if len(market_caps) >= min_names and total_mcap > 0:
            wr, vc = 0.0, 0
            for t, mcap in market_caps.items():
                try:
                    r = returns_df.loc[date, t]
                    if pd.notna(r):
                        wr += (mcap / total_mcap) * r
                        vc += 1
                except Exception:
                    pass
            ipo_ret = wr if vc >= min_names else np.nan
        else:
            ipo_ret = np.nan
        index_data.append({'date': date, 'ipo_ret': ipo_ret})
    return pd.DataFrame(index_data).set_index('date')

ipo_index = build_ipo_index_mcap(prices, ipo_df, shares_outstanding, holding_days=180)
print(f"IPO index: {ipo_index['ipo_ret'].notna().sum()} days with valid returns")

IPO index: 249 days with valid returns


In [6]:
# Build data dict: market_return (custom portfolio), ipo_return
ipo_ret = ipo_index['ipo_ret'].rename('ipo_return')

df = align_returns(market_ret, ipo_ret)
df = add_optional_features(df, include_vix=False)
feature_cols = list(df.columns)

WINDOW = 126  # ~6 months; 2025 has limited data
X, R, dates = build_rolling_windows(df, window_len=WINDOW, feature_cols=feature_cols)
X_train, R_train, d_train, X_val, R_val, d_val = train_val_split(X, R, dates, val_frac=0.2)

data = {
    "X_train": X_train, "R_train": R_train, "dates_train": d_train,
    "X_val": X_val, "R_val": R_val, "dates_val": d_val,
    "feature_cols": feature_cols, "df": df, "n_assets": 2, "window_len": WINDOW,
}
print(f"Train windows: {X_train.shape[0]}, Val windows: {X_val.shape[0]}")

Train windows: 99, Val windows: 24


In [7]:
# Train model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, history = run_training(
    data, device=device,
    epochs=50, lr=1e-3, batch_size=32, patience=10,
    model_type="gru",
)
print(f"Trained for {len(history)} epochs")

Trained for 11 epochs


In [8]:
# Predict and export
weights = predict_weights(model, data["X_val"], device)
stats = portfolio_stats(weights, data["R_val"])

out_dir = ROOT / "results"
out_dir.mkdir(exist_ok=True)
weights_path = out_dir / "ipo_optimizer_weights.csv"
summary_path = out_dir / "ipo_optimizer_summary.txt"

export_weights_csv(data["dates_val"], weights, weights_path)
export_summary(stats, weights, summary_path, R=data["R_val"])
print(f"Exported weights to {weights_path}")
print(f"Exported summary to {summary_path}")

Exported weights to C:\Users\ocean\anaconda_projects\STAT-4830-OSO\results\ipo_optimizer_weights.csv
Exported summary to C:\Users\ocean\anaconda_projects\STAT-4830-OSO\results\ipo_optimizer_summary.txt


In [9]:
# Policy interpretation
avg_ipo = float(weights[:, 1].mean()) if weights.shape[1] >= 2 else 0.0
scale = ipo_tilt_to_position_scale(avg_ipo)
print(policy_rule(avg_ipo))
print(f"Suggested position scale for next IPO: {scale:.2f}")
print(f"\nMetrics: Sharpe={stats['sharpe_annualized']:.2f}, MaxDD={stats['max_drawdown']:.2%}")

Consider increasing IPO exposure (model IPO weight > 20%).
Suggested position scale for next IPO: 0.75

Metrics: Sharpe=1.75, MaxDD=-3.07%
